# Lab 0: Environment Setup

**Objective**: Set up the development environment for building AI agents

In this lab, you will:
1. Install required Python packages
2. Set up GitHub Models for free LLM access
3. Deploy the mock backend using ngrok

---

## Step 1: Install Required Packages

We'll install all necessary Python packages for agent development.

> You may see an error message like this at the bottom, but that's okay, as long as the call is successfully excuted (there a green tick next to the cell on the left):
> ![image.png](https://i.ibb.co/nFVsYs4/python-package-download-error.png)

In [21]:
# Install all required packages
!pip install -q agent-framework --pre requests fastapi uvicorn pyngrok nest-asyncio

## Step 2: Configure GitHub Models Access with Colab Secrets

Instructions to Store Your GitHub Token Securely:

**Step 2a: Get Your GitHub Personal Access Token (PAT)**
1. Visit https://github.com/settings/tokens?type=beta
2. Click "Generate new token" (fine-grained token)
3. Set token name (e.g., "GitHub Models Access")
4. Set expiration (e.g., 90 days)
5. Under "Permissions", select "Read access to content"
6. Click "Generate token" and **COPY THE TOKEN**

**Step 2b: Store in Colab Secrets (Recommended)**
1. Look at the left sidebar of this Colab notebook
2. Click the **🔑 key icon** (Secrets)
3. Click **"+ Add new secret"**
4. Name: `GITHUB_PAT`
5. Value: **Paste your token here**
6. Toggle **"Notebook access"** to ON

![Colab Secrets Location](https://colab.research.google.com/img/colab_favicon_256px.png)

**Why use Secrets?**
- Tokens are encrypted and not visible in notebook cells
- Prevents accidental token exposure when sharing notebooks
- Follows security best practices

**Note**: GitHub Models provides free access to various LLMs for development purposes.

In [22]:
# Load GitHub token from Colab Secrets
from google.colab import userdata

try:
    GITHUB_PAT = userdata.get('GITHUB_PAT')
    print("✅ GitHub PAT loaded from Colab Secrets successfully!")
    print(f"✅ Token length: {len(GITHUB_PAT)} characters")
except Exception as e:
    print("⚠️ WARNING: Could not load GITHUB_PAT from Colab Secrets!")
    print("   Please follow Step 2b above to add your token to Secrets.")
    print(f"   Error: {e}")
    GITHUB_PAT = ""  # Fallback

✅ GitHub PAT loaded from Colab Secrets successfully!
✅ Token length: 93 characters


## Step 3: Test GitHub Models Connection

We'll test that we can connect to GitHub Models using the OpenAI Chat Completions API.

In [ ]:
from openai import OpenAI

In [ ]:
# Initialize OpenAI chat client with GitHub Models endpoint
client = OpenAI(
  base_url="https://models.github.ai/inference",
  api_key=GITHUB_PAT,
)

In [ ]:
completion = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in the meaning of life?"
                }
            ]
        }
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
)

In [23]:
print(completion.choices[0].message.content)

The meaning of life is a philosophical question that has been debated for centuries, and it can vary widely depending on individual beliefs, cultural backgrounds, and personal experiences. Here are a few perspectives:

1. **Philosophical Perspectives**: Different philosophical schools offer various interpretations. Existentialists, for example, argue that life has no inherent meaning, and it's up to individuals to create their own purpose. In contrast, utilitarianism suggests that the meaning of life lies in maximizing happiness and minimizing suffering.

2. **Religious Views**: Many religions provide specific teachings about the purpose of life. For instance, in Christianity, life may be seen as a journey to connect with God and live according to divine principles, while in Buddhism, the focus may be on achieving enlightenment and escaping the cycle of suffering.

3. **Personal Significance**: Many people find meaning through personal relationships, achievements, and contributions to 

## Step 4: Download & Deploy Mock Backend with ngrok

We'll download the FastAPI backend from the repository and deploy it using ngrok to create a public URL.

**What happens here:**
- Downloads `mock_backend.py` from GitHub to Colab's `/content/` directory (default working directory)
- Starts the FastAPI server in the background (runs on port 8000)
- Creates a public URL via ngrok for external access
- You can access the file at `/content/mock_backend.py` if needed

### Step 4a: Download & Save the Mock Backend

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import time
import requests

In [ ]:
# Allow nested event loops (required for Colab)
nest_asyncio.apply()

# Download the mock backend from GitHub
backend_url = "https://raw.githubusercontent.com/tezansahu/building-eval-driven-ai-agents/main/backend/mock_backend.py"

print("📥 Downloading mock backend from repository...")
response = requests.get(backend_url)

# Save to /content/ directory (Colab's default working directory)
with open('mock_backend.py', 'w') as f:
    f.write(response.text)

print("✅ Backend code downloaded to /content/mock_backend.py")

In [ ]:
# Import the backend module
import mock_backend

In [ ]:
# Start server in background thread
print("🚀 Starting FastAPI server...")
server_thread = mock_backend.run_in_thread(port=8000)
time.sleep(2)  # Give server time to start

print("✅ FastAPI server started on port 8000")

### Step 4b: Configure ngrok Authentication

**ngrok requires a free account and authtoken for usage.**

**Instructions:**
1. Go to https://dashboard.ngrok.com/signup and create a free account
2. After signup, go to https://dashboard.ngrok.com/get-started/your-authtoken
3. Copy your authtoken
4. In Colab, click the **🔑 key icon** (Secrets) on the left sidebar
5. Click **"+ Add new secret"**
6. Name: `NGROK_AUTHTOKEN`
7. Value: **Paste your ngrok authtoken**
8. Toggle **"Notebook access"** to ON

**Why ngrok?** It creates a public URL for our local backend so the agent can access it from anywhere.

In [31]:
try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("✅ ngrok authenticated successfully!")
except Exception as e:
    print("⚠️ WARNING: Could not load NGROK_AUTHTOKEN from Colab Secrets!")
    print("   Please follow the instructions above to add your ngrok authtoken.")
    print(f"   Error: {e}")
    raise

✅ ngrok authenticated successfully!


### Step 4c: Expose the Server Publicly

In [32]:
# Create public URL with ngrok
print("🌐 Creating public URL with ngrok...")
public_url = ngrok.connect(8000)
BACKEND_URL = public_url.public_url

print(f"\n{'='*60}")
print(f"✅ Backend server is running!")
print(f"{'='*60}")
print(f"📡 Public URL: {BACKEND_URL}")
print(f"📚 API Docs: {BACKEND_URL}/docs")
print(f"📂 File location: /content/mock_backend.py")
print(f"{'='*60}")
print(f"\n⚠️ IMPORTANT: Copy the Public URL above!")
print(f"   You'll need it for Lab 1 & Lab 2")

🌐 Creating public URL with ngrok...

✅ Backend server is running!
📡 Public URL: https://26ffc6efa184.ngrok-free.app
📚 API Docs: https://26ffc6efa184.ngrok-free.app/docs
📂 File location: /content/mock_backend.py

⚠️ IMPORTANT: Copy the Public URL above!
   You'll need it for Lab 1 & Lab 2


## Step 5: Test Backend API

Let's verify the backend is working correctly by testing the endpoints.

In [35]:
import requests
import json

In [36]:
# Test 1: Health check
response = requests.get(f"{BACKEND_URL}/")
print("Health Check:", response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [28]:
# Test 2: List events
response = requests.get(f"{BACKEND_URL}/events")
events = response.json()
print(f"\nFound {len(events)} events.:")
for event in events:
    print(f"  - {event['name']} on {event['date']}")

NameError: name 'BACKEND_URL' is not defined

## 🎉 Lab 0 Complete!



### What You Accomplished:

✅ **Installed Microsoft Agents Framework** - A high-level library for building AI agents  
✅ **Configured GitHub Models** - Free LLM access using GitHub PAT  
✅ **Stored secrets securely** - Using Colab Secrets (best practice)  
✅ **Deployed mock backend** - Campus event management API with ngrok  
✅ **Tested the setup** - Verified agent can call LLM and backend APIs

### Key Concepts Learned:

1. **Colab Secrets**: Secure way to store API tokens without exposing them in code
2. **GitHub Models**: Free access to GPT-4o-mini for development
3. **ngrok**: Makes local services accessible via public URL

### What's Next?

**Lab 1**: Build a complete campus event agent with 3 tools:
- Register students for events
- Book venues for clubs
- Send notifications to participants

**Lab 2**: Evaluate and improve the agent using metrics:
- Relevance Evaluator (Azure AI)
- Task Adherence Evaluator (Azure AI)
- Custom Conciseness Evaluator
- Measure improvement systematically

---


## Troubleshooting:

**Issue**: "Could not load GITHUB_PAT from Colab Secrets"
- **Solution**: Click 🔑 icon → Add new secret → Name: `GITHUB_PAT` → Enable notebook access

**Issue**: "ngrok tunnel failed"
- **Solution**: Re-run the ngrok cell. Tunnels expire after inactivity.

**Issue**: Backend API not responding
- **Solution**: Check that BACKEND_URL is set correctly and starts with `https://`